In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import time
from scipy import stats

In [11]:
# Uniform distribution
def gen_uniform(m,a,b,N,x0):
    result = []
    x = x0
    for i in range(N):
        x = (a * x + b) % m # choose method here
        result.append(x / m)  # Normalize to range [0, 1]
    return np.array(result)

In [28]:
# Binomial distribution
# Simulate 44 x 1000 uniform distribution
# 1000 random numbers with binomial distribution n = 44 and p = 0.64
# Bernulli(n,p) = sum of 44 Bernulli(p)
U1 = pd.DataFrame(gen_uniform(m=2**31-1, a=7**5, b=0, N=44_000, x0=123).reshape((1000,44)))

# Define each simulated item to be one Bernulli(p=0.64) and sum 44 Bernulli rv. to form a Binomial rv
# Each Column in a Bernulli; sum across column result in 1000 Bin(n,p)
X1 = (U1 <= 0.64).sum(axis=1)
# Calculate probability that X is greater than 35
Prob_Bin = (X1 >= 35).mean()

## Empirical number is 0.02
Diff = Prob_Bin - 0.02
print('Probbaility:',Prob_Bin, 'Difference:', Diff)

Probbaility: 0.016 Difference: -0.004


In [12]:
# Assumed LGM method for uniform 
# F is invertible to empliment this method
def exp_gen(lam):
    U1 = gen_uniform(m=2**31-1, a=7**5, b=0, N=10_000, x0=2)
    # inverse F (cdf of exponent)
    X = - lam * np.log(1-U1)
    Empirical_Std = X.std()/np.sqrt(10_000-1)
    Empirical_mean = X.mean()
    return [X,Empirical_Std,Empirical_mean]

# Probability of X>=1:
Prob_exp_1 = (exp_gen(lam=1.5)[0]>=1).mean()


1.5170521563846937

In [23]:
# Generate Normal with Box-Muller
def Normal_BM(N_1,mean,std):
    U1 = gen_uniform(m=2**31-1, a=7**5, b=0, N=int(N_1/2), x0=3)
    U2 = gen_uniform(m=2**31-1, a=7**5, b=0, N=int(N_1/2), x0=4)
    Z1 = mean + std * np.concatenate(([np.sqrt(-2*np.log(U1)) * np.cos(2*np.pi*U2),np.sqrt(-2*np.log(U1)) * np.sin(2*np.pi*U2)]))
    return Z1

In [59]:
# Generate Normal with Polar-Marsaglia
def Normal_PM(N_1,mean,std):
    # Generate (10_000>5000) to make sure enough w for Z
    U = 2 * gen_uniform(m=2**31-1, a=7**5, b=0, N=10000, x0=1) - 1
    df = pd.DataFrame(U.reshape((5000,2)), columns=['U1', 'U2'])
    df['W'] = df['U1']**2+df['U2']**2
    df = df[df['W']<1][:2500]
    df['U1'] = df['U1'] * np.sqrt((-2 * np.log(df['W']))/df['W'])
    df['U2'] = df['U2'] * np.sqrt((-2 * np.log(df['W']))/df['W'])
    result = mean + std * np.concatenate((df['U1'], df['U2']))
    return result